***GENERATED CODE FOR joinwithdataprep PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
from pyspark.sql import functions as F


def QueryBuilder(spark, joindatalist, paths):
    listOfDfnames = {}
    for path in paths:
        listOfDfnames[path['sourcename']] = spark.read.options(
            header='true', inferschema='true', delimiter=',').csv('dbfs:' + path['sourcepath'])
    for join in joindatalist:
        dfA = listOfDfnames[join['sourceA']['sourcename']]
        dfB = listOfDfnames[join['sourceB']['sourcename']]
        f1 = 'a.' + join['sourceA']['fieldname'].replace(" ", "")
        f2 = 'b.' + join['sourceB']['fieldname'].replace(" ", "")
        listOfDfnames[join['sourceA']['sourcename'] + '_' + join['sourceB']['sourcename']] = removeIfDuplicates(dfA.alias('a').join(
            dfB.alias('b'), F.col(f1) == F.col(f2),
            how=join['joinName']))
        joinedDf = listOfDfnames[join['sourceA']
                                 ['sourcename'] + '_' + join['sourceB']['sourcename']]
    df = removeIfDuplicates(joinedDf)
    print(display(df.limit(2).toPandas()))
    return df


def removeIfDuplicates(joinedDf):
    cols_new = []
    seen = set()
    for c in joinedDf.columns:
        cols_new.append('{}_dup'.format(c) if c in seen else c)
        seen.add(c)
    df = joinedDf.toDF(*cols_new).select(*
                                         [c for c in cols_new if not c.endswith('_dup')])
    return df


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'price_sum', 'threshold': 162125.88109360123, 'transformation_label': 'Binarizer'}], 'feature': 'price_sum', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '500', 'mean': '162125.88', 'stddev': '959787.81', 'min': '117.19', 'max': '1.209917605204711E7', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'price_sum'}, {'feature_label': 'price_sum', 'threshold': 162125.88109360123, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('price_sum')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'start_date', 'transformation_label': 'Extract Date'}], 'feature': 'start_date', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'start_date'}, {'feature_label': 'start_date', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('start_date')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'end_date', 'transformation_label': 'Extract Date'}], 'feature': 'end_date', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'end_date'}, {'feature_label': 'end_date', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('end_date')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
#%run joinwithdataprepHooks.ipynb
try:
	#sourcePreExecutionHook()
		coupon = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/Data/TrainData/couponredemption/coupon.csv', 'filename': 'coupon.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line'}")


		hdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/Data/TrainData/couponredemption/cust_demo.csv', 'filename': 'cust_demo.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line'}")


		train = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/Data/TrainData/couponredemption/train.csv', 'filename': 'train.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line'}")


		transactions = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/Data/TrainData/couponredemption/transactions.csv', 'filename': 'transactions.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line'}")



	campaign = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/Data/TrainData/couponredemption/campaign.csv', 'filename': 'campaign.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(campaign)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run joinwithdataprepHooks.ipynb
try:
	#operationPreExecutionHook()

	join = QueryBuilder(spark,[{"sourceA": {"sourceid": "5fe46269c4cf4870a28b7d35", "fieldname": "coupon_id", "sourcepath": "/FileStore/platform/Data/TrainData/couponredemption/train.csv", "sourcename": "train"}, "sourceB": {"sourceid": "5fe46261c4cf4870a28b7d32", "fieldname": "coupon_id", "sourcepath": "/FileStore/platform/Data/TrainData/couponredemption/coupon.csv", "sourcename": "coupon"}, "joinName": "inner"}, {"sourceA": {"sourceid": "5fe46269c4cf4870a28b7d35_5fe46261c4cf4870a28b7d32", "fieldname": "campaign_id", "sourcepath": "", "sourcename": "train_coupon"}, "sourceB": {"sourceid": "5fe4626dc4cf4870a28b7d37", "fieldname": "campaign_id", "sourcepath": "/FileStore/platform/Data/TrainData/couponredemption/campaign.csv", "sourcename": "campaign"}, "joinName": "inner"}, {"sourceA": {"sourceid": "5fe46269c4cf4870a28b7d35_5fe46261c4cf4870a28b7d32_5fe4626dc4cf4870a28b7d37", "fieldname": "customer_id", "sourcepath": "", "sourcename": "train_coupon_campaign"}, "sourceB": {"sourceid": "5fe46263c4cf4870a28b7d34", "fieldname": "customer_id", "sourcepath": "/FileStore/platform/Data/TrainData/couponredemption/cust_demo.csv", "sourcename": "HDFS"}, "joinName": "inner"}, {"sourceA": {"sourceid": "5fe46269c4cf4870a28b7d35_5fe46261c4cf4870a28b7d32_5fe4626dc4cf4870a28b7d37_5fe46263c4cf4870a28b7d34", "fieldname": "customer_id", "sourcepath": "", "sourcename": "train_coupon_campaign_HDFS"}, "sourceB": {"sourceid": "5fe4626bc4cf4870a28b7d36", "fieldname": "customer_id", "sourcepath": "/FileStore/platform/Data/TrainData/couponredemption/transactions.csv", "sourcename": "transactions"}, "joinName": "inner"}],[{'sourcename': 'train', 'sourcepath': '/FileStore/platform/Data/TrainData/couponredemption/train.csv'}, {'sourcename': 'coupon', 'sourcepath': '/FileStore/platform/Data/TrainData/couponredemption/coupon.csv'}, {'sourcename': 'campaign', 'sourcepath': '/FileStore/platform/Data/TrainData/couponredemption/campaign.csv'}, {'sourcename': 'HDFS', 'sourcepath': '/FileStore/platform/Data/TrainData/couponredemption/cust_demo.csv'}, {'sourcename': 'transactions', 'sourcepath': '/FileStore/platform/Data/TrainData/couponredemption/transactions.csv'}])
	#operationPostExecutionHook(join)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run joinwithdataprepHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(join,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "14450.4", "stddev": "8321.17", "min": "15", "max": "28186", "missing": "0"}, "updatedLabel": "id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "14.15", "stddev": "8.21", "min": "1", "max": "30", "missing": "0"}, "updatedLabel": "campaign_id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "573.55", "stddev": "334.68", "min": "1", "max": "1114", "missing": "0"}, "updatedLabel": "coupon_id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "customer_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "749.96", "stddev": "468.16", "min": "6", "max": "1581", "missing": "0"}, "updatedLabel": "customer_id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "redemption_status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.13", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "redemption_status"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "brand", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1180.19", "stddev": "1448.31", "min": "5", "max": "5391", "missing": "0"}, "updatedLabel": "brand"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "brand_type", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.21", "stddev": "0.4", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "brand_type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "category", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "7.47", "stddev": "2.52", "min": "1", "max": "16", "missing": "0"}, "updatedLabel": "category"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "cd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-1082.43", "stddev": "4778.13", "min": "-70597.67999999992", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "cd_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_discount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-1.89", "stddev": "4.4", "min": "-53.31", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "coupon_discount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_used", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "34.73", "stddev": "171.26", "min": "0.0", "max": "2581.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "coupon_used"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "item_counts", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "48.46", "stddev": "289.74", "min": "1", "max": "3718", "missing": "0"}, "updatedLabel": "item_counts"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_customers", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "17.99", "stddev": "13.91", "min": "1.0", "max": "118.57142857142857", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "no_of_customers"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "od_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-27472.17", "stddev": "162641.63", "min": "-2195914.9921420026", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "od_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "other_discount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-18.8", "stddev": "16.8", "min": "-86.88852770814994", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "other_discount"}, {"transformationsData": [{"feature_label": "price_sum", "threshold": 162125.88109360123, "transformation_label": "Binarizer"}], "feature": "price_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "162125.88", "stddev": "959787.81", "min": "117.19", "max": "1.209917605204711E7", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "price_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "qu_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2121.58", "stddev": "14258.48", "min": "1.0", "max": "207579.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "qu_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "quantity", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.18", "stddev": "0.25", "min": "1.0", "max": "2.8748556998557", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "selling_price", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "156.47", "stddev": "141.69", "min": "14.24791543142704", "max": "1235.863616557734", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "selling_price"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "t_counts", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1653.38", "stddev": "10921.2", "min": "1.0", "max": "158788.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "t_counts"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_discount_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-20.69", "stddev": "17.83", "min": "-100.70658333333334", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_discount_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_discount_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-28554.6", "stddev": "166804.88", "min": "-2216185.982142001", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_discount_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign_type", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.31", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "campaign_type"}, {"transformationsData": [{"feature_label": "start_date", "transformation_label": "Extract Date"}], "feature": "start_date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "start_date"}, {"transformationsData": [{"feature_label": "end_date", "transformation_label": "Extract Date"}], "feature": "end_date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "end_date"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign_duration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "44.57", "stddev": "6.32", "min": "32", "max": "61", "missing": "0"}, "updatedLabel": "campaign_duration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "age_range", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.4", "stddev": "1.24", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "age_range"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "marital_status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.4", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "marital_status"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rented", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.05", "stddev": "0.21", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "rented"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "family_size", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.25", "stddev": "1.23", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "family_size"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_children", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.64", "stddev": "1.0", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "no_of_children"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "income_bracket", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "4.75", "stddev": "2.28", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "income_bracket"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "day", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "15.53", "stddev": "8.24", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "day"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dow", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "3.2", "stddev": "1.65", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "dow"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_items", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "673.93", "stddev": "320.71", "min": "97", "max": "2088", "missing": "0"}, "updatedLabel": "no_of_items"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "4.79", "stddev": "2.14", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "cdd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-872.57", "stddev": "1531.27", "min": "-11937.8", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "cdd_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "customer_id_count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1207.65", "stddev": "654.52", "min": "144", "max": "4134", "missing": "0"}, "updatedLabel": "customer_id_count"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "odd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-16173.18", "stddev": "9436.18", "min": "-62321.37241", "max": "-2029.067149", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "odd_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "qa_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "182259.65", "stddev": "263609.82", "min": "206", "max": "2617544", "missing": "0"}, "updatedLabel": "qa_sum"}, {"feature": "price_sum_binarizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "0.14", "stddev": "0.35", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "price_sum_binarizer_trans..."}, {"feature": "start_date_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "15.84", "stddev": "4.74", "min": "2", "max": "28", "missing": "0"}, "updatedLabel": "start_date_dayofmonth"}, {"feature": "start_date_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "4.93", "stddev": "3.14", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "start_date_month"}, {"feature": "start_date_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2012.76", "stddev": "0.43", "min": "2012", "max": "2013", "missing": "0"}, "updatedLabel": "start_date_year"}, {"feature": "end_date_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "9.09", "stddev": "7.44", "min": "1", "max": "30", "missing": "0"}, "updatedLabel": "end_date_dayofmonth"}, {"feature": "end_date_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "5.31", "stddev": "2.63", "min": "1", "max": "11", "missing": "0"}, "updatedLabel": "end_date_month"}, {"feature": "end_date_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2012.87", "stddev": "0.33", "min": "2012", "max": "2013", "missing": "0"}, "updatedLabel": "end_date_year"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run joinwithdataprepHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["id", "campaign_id", "coupon_id", "customer_id", "brand", "brand_type", "category", "cd_sum", "coupon_discount", "coupon_used", "item_counts", "no_of_customers", "od_sum", "other_discount", "price_sum_binarizer", "qu_sum", "quantity", "selling_price", "t_counts", "total_discount_mean", "total_discount_sum", "campaign_type", "campaign_duration", "age_range", "marital_status", "rented", "family_size", "no_of_children", "income_bracket", "day", "dow", "no_of_items", "month", "cdd_sum", "customer_id_count", "odd_sum", "qa_sum", "start_date_dayofmonth", "start_date_month", "start_date_year", "end_date_dayofmonth", "end_date_month", "end_date_year"], "redemption_status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

